# seq2seqCalculator

## Import requirements

In [96]:
from random import seed
from random import randint

import numpy as np

from sklearn.preprocessing import LabelBinarizer 


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

from keras.callbacks import ModelCheckpoint
import tensorflow as tf


import os

## Data
One benefit of this task is that you don't need to download any data. You can generate it on your own! The input consists of the following: "number"+"sign"+"number", which then equals "number". The pluses stand for concatenation in this case. 
The seq2seq model doesn't care what symbols we use, we could use "+-*/" as symbols. 
<!-- We use a one-hot encoding for each symbol accroding the assigned number. For numbers this is straight forward and we end up with something like {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8: -->

In [97]:
def generate_equations(operators, dataset_size, min_value=0, max_value=10):
    '''Generate pairs of equations and solutions to them.
        
        Each equation has a form of two integers with an operator in between. 
        Each solution is an integer with the result of the operation.
        
        
        operators: list of strings, allowed operators. 
        dataset_size: an integer, number of equations to be generated.
        min_value: an integer, min value of each operand.
        max_value: an integer, max value of each operand.
    
        result: a list of tuples of strings (equation, solution).
    '''
    samples = []
    number_of_operators = len(operators)
    
    for _ in range(dataset_size):
        equation = (
            str(randint(min_value, max_value)) + 
            operators[randint(0, number_of_operators-1)] +
            str(randint(min_value, max_value))
        )
        
        try:
            solution = str(int(eval(equation)))
        except ZeroDivisionError as e: ## handle x/0 state
            equation = equation.replace("/", "*")
            solution = str(int(eval(equation)))

        samples.append((equation, solution))
    
    return samples

In [73]:
## Test generate_equations
test = generate_equations("+-/*",1)
print(test)

[('1*5', '5')]


## Perpare data for the neural network

The maximum input length is “length_nr + 1 + length_nr” which is ```x_max_length``` in our case (for example 7). We would like to also have calculations that are shorter. This is entirely possible, however a seq2seq requires [check] inputs of the same length. So when a calculation is smaller then length ```x_max_length``` we fill it up from the left with spaces, e.g. “____2+2”.
Also we need to padd the solutions to ```y_max_length```.

In [98]:
def padding_to_max(equations, x_max_length, y_max_length):
    '''Padd the equations to max lengths.
    
    equations: a list of tuples of strings (equation, solution)
    x_max_length: max_len of equation samples.
    y_max_length: max_len of solution samples.
    
    return:  a list of tuples of strings (equation, solution) that padded to max length.
    '''
    samples = []
    for equation, solution in equations:
        samples.append((f'{equation:>{x_max_length}}', f'{solution:>{y_max_length}}'))
    return samples

In [75]:
def to_ids(equations, word2id):
    '''Convert equations to ids. 
    '''
    samples = []
    for equation, solution in equations:
        e = [word2id[c] for c in equation]
        s = [word2id[c] for c in solution]
        samples.append((e, s))
    return samples

In [5]:
def one_hot(equations, labels):
    '''Create one-hot encoder for equations.
    '''
    label_binarizer = LabelBinarizer() 
    label_binarizer.fit(labels)
    x, y = [], []
    for equation, solution in equations:
        x.append(label_binarizer.transform(equation))
        y.append(label_binarizer.transform(solution))
    return x, y

In [6]:
# invert encoding
def invert(seq, id2word):
    strings = list()
    for pattern in seq:
        string = id2word[np.argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [119]:
def generate_data(n_samples, allowed_operators, alphabet, word2id, id2word, x_max_length=7, y_max_length=4, min_value=0, max_value=100):
    ## generate pairs
    data = generate_equations(allowed_operators, n_samples, min_value=min_value, max_value=max_value)    

    ## padding to max
    data = padding_to_max(data, x_max_length, y_max_length)
    
    ## string to indexs
    data = to_ids(data, word2id)
    # one hot encoding 
    x, y = one_hot(data, list(id2word.keys()))

    x, y = np.array(x), np.array(y)
    
    return x, y


### Setup variables

In this section we should set the number of samples, allowed opeatots and alphabet:

In [143]:
seed(1)

allowed_operators = '+'
n_samples = 1000
alphabet = '0123456789' + allowed_operators + ' ' ## space for padding

word2id = {symbol:i for i, symbol in enumerate(alphabet)}
id2word = {i:symbol for symbol, i in word2id.items()}

x_max_length, y_max_length = 7, 4
min_value, max_value = 0, 100

x, y = generate_data(n_samples, allowed_operators, alphabet, word2id, id2word, x_max_length=x_max_length, y_max_length=y_max_length, 
                    min_value=min_value, max_value=max_value)

Test the functions:

In [144]:
print("shape of X", x.shape)
print("shape of y", y.shape)
print("X[0]:")
print(x[0])
print("y[0]")
print(y[0])
print("invert X[0]", invert(x[0], id2word) )
print("invert y[0]", invert(y[0], id2word) )

shape of X (1000, 7, 12)
shape of y (1000, 4, 12)
X[0]:
[[0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]]
y[0]
[[0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]]
invert X[0]   17+32
invert y[0]   49


## Model

In [145]:
# define LSTM configuration
n_batch = 10
n_epoch = 50
n_chars = len(alphabet)

In [146]:
# create LSTM
def build_model(x_max_length, y_max_length, n_chars):
    model = Sequential()
    model.add(LSTM(100, input_shape=(x_max_length, n_chars)))
    model.add(RepeatVector(y_max_length))
    model.add(LSTM(50, return_sequences=True))
    model.add(TimeDistributed(Dense(n_chars, activation='softmax')))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model


### Configure checkpoints

Use a ```tf.keras.callbacks.ModelCheckpoint``` to ensure that checkpoints are saved during training:


In [147]:
# Directory where the checkpoints will be saved
checkpoint_dir = './models/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Train the model:

In [148]:
# train LSTM
model = build_model(x_max_length, y_max_length, n_chars)

print(model.summary())

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_54 (LSTM)              (None, 100)               45200     
                                                                 
 repeat_vector_27 (RepeatVec  (None, 4, 100)           0         
 tor)                                                            
                                                                 
 lstm_55 (LSTM)              (None, 4, 50)             30200     
                                                                 
 time_distributed_27 (TimeDi  (None, 4, 12)            612       
 stributed)                                                      
                                                                 
Total params: 76,012
Trainable params: 76,012
Non-trainable params: 0
_________________________________________________________________
None


In [154]:
for i in range(n_epoch):

    x, y = generate_data(n_samples, allowed_operators, alphabet, word2id, id2word, x_max_length=x_max_length, y_max_length=y_max_length, 
                    min_value=min_value, max_value=max_value)
    print(i)
    model.fit(x, y, epochs=1, batch_size=n_batch, callbacks=[checkpoint_callback])

0
100/100 [==============================] - 8s 11ms/step - loss: 0.0195 - accuracy: 0.8112
1
100/100 [==============================] - 1s 10ms/step - loss: 0.0192 - accuracy: 0.8223
2
100/100 [==============================] - 1s 10ms/step - loss: 0.0189 - accuracy: 0.8177
3
100/100 [==============================] - 1s 7ms/step - loss: 0.0198 - accuracy: 0.8092
4
100/100 [==============================] - 1s 7ms/step - loss: 0.0201 - accuracy: 0.8095
5
100/100 [==============================] - 1s 7ms/step - loss: 0.0194 - accuracy: 0.8160
6
100/100 [==============================] - 1s 8ms/step - loss: 0.0191 - accuracy: 0.8167
7
100/100 [==============================] - 1s 7ms/step - loss: 0.0180 - accuracy: 0.8345
8
100/100 [==============================] - 1s 7ms/step - loss: 0.0174 - accuracy: 0.8420
9
100/100 [==============================] - 1s 7ms/step - loss: 0.0185 - accuracy: 0.8332
10
100/100 [==============================] - 1s 7ms/step - loss: 0.0182 - accuracy: 0.

In [150]:
tf.train.latest_checkpoint(checkpoint_dir)

'./models/training_checkpoints\\ckpt_1'

In [155]:
model = build_model(x_max_length, y_max_length, n_chars)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# evaluate on some new patterns
# x, y = generate_data(n_samples, allowed_operators, alphabet, word2id, id2word)    
x, y = generate_data(n_samples, allowed_operators, alphabet, word2id, id2word, x_max_length=x_max_length, y_max_length=y_max_length, 
                    min_value=min_value, max_value=max_value)

result = model.predict(x, batch_size=n_batch, verbose=0)

# calculate error
expected = [invert(x, alphabet) for x in y]
predicted = [invert(x, alphabet) for x in result]

# show some examples
for i in range(20):
    print('%s Expected=%s, Predicted=%s' % (invert(x[i], id2word),expected[i], predicted[i]))

  94+49 Expected= 143, Predicted= 143
   62+3 Expected=  65, Predicted=  65
  80+39 Expected= 119, Predicted= 118
  26+83 Expected= 109, Predicted= 109
  77+79 Expected= 156, Predicted= 156
  88+51 Expected= 139, Predicted= 139
  61+33 Expected=  94, Predicted=  94
  69+48 Expected= 117, Predicted= 117
  35+78 Expected= 113, Predicted= 113
  76+73 Expected= 149, Predicted= 149
  55+35 Expected=  90, Predicted=  90
  56+65 Expected= 121, Predicted= 121
  76+11 Expected=  87, Predicted=  87
   57+1 Expected=  58, Predicted=  58
  54+89 Expected= 143, Predicted= 143
   21+5 Expected=  26, Predicted=  27
  60+49 Expected= 109, Predicted= 108
   99+0 Expected=  99, Predicted=  98
  99+79 Expected= 178, Predicted= 178
   50+6 Expected=  56, Predicted=  56
